In [20]:
# Install selenium
# ! pip install selenium
# Import the libraries.
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import urllib
import time
import os

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

HOME = os.getcwd()

C:\Users\bouab\AppData\Local\Temp\ipykernel_12880\3929526215.py:15: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


In [21]:
# # Create url variable containing the webpage for a Google image search.
# url = ("https://www.google.com/search?q={s}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568")
# # Launch the browser and open the given url in the webdriver.
# driver.get(url.format(s='Pets'))
# # Scroll down the body of the web page and load the images.
# driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
# time.sleep(5)
# # Find the images.
# imgResults = driver.find_elements(By.XPATH,"//img[contains(@class,'Q4LuWd')]")
# # Access and store the scr list of image url's.
# src = []
# for img in imgResults:
#     src.append(img.get_attribute('src'))
# # Retrieve and download the images.
# for i in range(10):    urllib.request.urlretrieve(str(src[i]),"sample_data/pets{}.jpg".format(i))

In [22]:
def scrape_images(driver: webdriver, class_name: str, search_query: str, file_location: str=None, limit: int=200, wait_time:int=30):
    # first define url
    url = ("https://www.google.com/search?q={s}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568")
    # set the wait time for the driver
    driver.implicitly_wait(wait_time)
    
    try:
        driver.get(url.format(s=search_query))

        img_results = []        

        for i in range(10):
            # Scroll down the body of the web page and load the images.
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            time.sleep(10)
            # Find the images.
            img_results.extend(driver.find_elements(By.XPATH,"//img[contains(@class,'Q4LuWd')]"))
        
        print(f"found {len(img_results)} images for {search_query}")

        if file_location is None:
            file_location = os.path.join(os.getcwd(), 'data_selenium', class_name)
            os.makedirs(file_location, exist_ok=True)

        
        for i, image in enumerate(img_results):                
            # certain images are not associated with a link, which raises an error, adding a try catch block here maximizes the number of scrapped images
            try:
                urllib.request.urlretrieve(str(image.get_attribute('src')), os.path.join(file_location, f'{search_query}_image_{i}.jpg'))
            except Exception as e:
                pass
            
    except Exception as e:
        print(f"Something wrong went with scraping {search_query}")
        print(e)



In [23]:
queries = [['кухонный шкаф напольный', 'кухонный шкаф навесной', 'кухонный шкаф икеа', 'кухонный шкаф новый',
            'кухонный шкаф с выдвижными ящиками','кухонный шкаф пенал'], 
           ['bathtub', 'bathtub new', 'bathtub modern', 'bathtub big', 'bathtub small'],
           ['electrical switch on a wall', 'electrical switch white', 'electrical switch new', 'light switches','light switches on a wall'], 
           ['electrical socket', 'electrical socket on a wall', 'electrical outlet', 'electrical outlet on a wall', 'outlet for charger'], 
           ['sink', 'sink modern', 'sink new', 'sink big', 'sink kitchen'], 
           ['door', 'door new', 'door modern', 'door big', 'door small'],
           ['toilet seat', 'toilet seat modern', 'toilet seat small', 'toilet seat new', 'toilet seat big']]


classes = ['kitchen', 'bath', 'switch', 'socket', 'sink', 'door', 'toilet seat']

# avoid running this piece of code unless the directo
DATA_DIR = os.path.join(HOME, 'data_selenium')

if not os.path.isdir(DATA_DIR) or len(os.listdir(os.path.join(HOME, 'data_selenium'))) == 0:
    for c, qs in zip(classes[1:], queries[1:]):
        for query in qs:
            scrape_images(driver, c, search_query=query, wait_time=60)
            print(f"scrapping for {query} done !!!")
            print()
        print(f"SCRAPPING FOR CLASS {c} DONE!!!")
        print("\n"*3)

The data  has a large number of duplicate images, Manual filtering does not seem like a reasonable option taking into account the size of the scrapped data. Thus, image hasing comes to the rescue.   
The Wavelet Hashing technique was used as it is does a better job detecting duplicates than the rest of the hashing techniques.

In [24]:
from PIL import Image
import imagehash as ih
import numpy as np

c = classes[-1]

# first create the path variable
dir_path = os.path.join(DATA_DIR, c)
# get all the file names 
images = [os.path.join(dir_path, file_name) for file_name in os.listdir(dir_path)]
i = images[0]
h = ih.whash(Image.open(i))


In [25]:
# time to filter the images

for c in classes:
    # first create the path variable
    dir_path = os.path.join(DATA_DIR, c)
    # create a set to store the unique hashes
    unique_hashes = set()
    # iterate through each image
    images = [os.path.join(dir_path, file_name) for file_name in os.listdir(dir_path)]
    for i in images:
        h = ih.whash(Image.open(i))
        if h not in unique_hashes:
            unique_hashes.add(h)
        else:
            # the image should be removed as it is a duplicated (or very similar to another existing image)
            os.remove(i)
    